<a href="https://colab.research.google.com/github/adityasingh345/Music_genre_classifier/blob/main/Music_genre_classifier_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "/content/drive/MyDrive/Data.zip" -d "/content/data/"

Archive:  /content/drive/MyDrive/Data.zip
  inflating: /content/data/features_30_sec.csv  
  inflating: /content/data/features_3_sec.csv  
  inflating: /content/data/genres_original/blues/blues.00000.wav  
  inflating: /content/data/genres_original/blues/blues.00001.wav  
  inflating: /content/data/genres_original/blues/blues.00002.wav  
  inflating: /content/data/genres_original/blues/blues.00003.wav  
  inflating: /content/data/genres_original/blues/blues.00004.wav  
  inflating: /content/data/genres_original/blues/blues.00005.wav  
  inflating: /content/data/genres_original/blues/blues.00006.wav  
  inflating: /content/data/genres_original/blues/blues.00007.wav  
  inflating: /content/data/genres_original/blues/blues.00008.wav  
  inflating: /content/data/genres_original/blues/blues.00009.wav  
  inflating: /content/data/genres_original/blues/blues.00010.wav  
  inflating: /content/data/genres_original/blues/blues.00011.wav  
  inflating: /content/data/genres_original/blues/blues.00

In [ ]:
import os

In [ ]:
folder = '/content/data/genres_original'  # or the path in Drive if you used Option A
print("Genres:", os.listdir(folder))

Genres: ['hiphop', 'disco', 'blues', 'metal', 'reggae', 'jazz', 'rock', 'classical', 'pop', 'country']


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install librosa soundfile audiomentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.5/86.5 kB 863.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 2.9 MB/s eta 0:00:00


In [ ]:
import librosa

In [ ]:
# now create the function to extract all the features of every song
import numpy as np
import librosa

def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, duration=30)

        # Proceed only if y was loaded
        if y is None or len(y) == 0:
            print(f"❌ Empty or unreadable audio: {file_path}")
            return None

        # Extract Features
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)


        # Feature Means
        mfcc_mean = np.mean(mfcc, axis=1).flatten()               # (13,)
        chroma_mean = np.mean(chroma, axis=1).flatten()           # (12,)
        zcr_mean = np.mean(zcr).flatten()                         # (1,)
        zcr_std = np.std(zcr).flatten()                           # (1,)
        spec_contrast_mean = np.mean(spec_contrast, axis=1).flatten()  # (7,)
        tempo = np.array([tempo]).flatten()

        # Combine all features into a single vector
        feature_vector = np.hstack([
            mfcc_mean,
            chroma_mean,
            zcr_mean,
            zcr_std,
            spec_contrast_mean,
            tempo
        ])

        return feature_vector

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")
        return None

dataset_path = '/content/data/genres_original'

X = []
Y = []

genres = sorted(os.listdir(dataset_path))  # Ensures consistent label order

total = 0
success = 0
failures = []

for genre in genres:
    genre_path = os.path.join(dataset_path, genre)
    if not os.path.isdir(genre_path):
        continue
    print(f"Processing genre: {genre}")

    for filename in os.listdir(genre_path):
        if filename.endswith(".wav"):
            total += 1
            file_path = os.path.join(genre_path, filename)
            features = extract_features(file_path)
            if features is not None:
                X.append(features)
                Y.append(genre)
                success += 1
            else:
                failures.append(file_path)

# Filter out None values from X and Y after the loop
X = [f for f in X if f is not None]
Y = [y for y, f in zip(Y, X) if f is not None] # Keep only labels for successful feature extractions

Processing genre: blues
Processing genre: classical
Processing genre: country
Processing genre: disco
Processing genre: hiphop
Processing genre: jazz


/tmp/ipython-input-703582430.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, duration=30)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


❌ Error processing /content/data/genres_original/jazz/jazz.00054.wav: 
Processing genre: metal
Processing genre: pop
Processing genre: reggae
Processing genre: rock


In [ ]:
 lengths = [len(f) for f in X]
print(set(lengths))  # This shows all unique lengths

{35}


In [ ]:
X = np.array(X)
Y = np.array(Y)
print(X.shape)  # Should be (num_songs, 35)

(999, 35)


In [ ]:
df = pd.DataFrame(X)

In [ ]:
df['label'] = Y

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,label
0,-255.973831,110.827522,21.460573,67.588135,4.288312,43.118462,-4.508348,10.442597,-8.718592,1.654890,...,0.027574,25.499474,19.092094,20.626000,17.990107,19.863146,18.939434,35.851431,123.046875,blues
1,-52.701164,74.455414,-18.715458,56.307217,-3.959327,31.987156,-16.457788,22.417690,-12.262029,27.228352,...,0.051619,19.153947,16.769936,21.166412,21.773673,21.574687,19.244753,44.478021,117.453835,blues
2,-118.370331,74.867317,-19.869467,46.947586,-32.569054,23.835136,-26.374111,17.692514,-13.826244,11.946739,...,0.050530,20.249789,16.411930,20.794435,20.532554,20.034756,19.510043,41.900797,107.666016,blues
3,-90.707222,100.107582,-23.924427,52.588303,-20.928015,31.502182,-11.213113,20.094328,-10.915236,17.563923,...,0.049328,19.394646,14.241172,16.221734,17.508593,18.277916,18.680311,41.277834,123.046875,blues
4,-163.243134,89.784889,12.977459,80.692482,3.666775,39.745129,-5.400379,26.498671,-13.163736,2.707013,...,0.016320,24.200335,15.850984,19.495703,17.599240,19.718683,17.326423,39.332214,129.199219,blues


In [ ]:
columns = [f'mfcc_{i}' for i in range(13)] + \
          [f'chroma_{i}' for i in range(12)] + \
          ['zcr_mean', 'zcr_std'] + \
          [f'spec_contrast_{i}' for i in range(7)] + \
          ['tempo']
df = pd.DataFrame(X, columns=columns)
df['label'] = Y

In [ ]:
df.head()

,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,...,zcr_std,spec_contrast_0,spec_contrast_1,spec_contrast_2,spec_contrast_3,spec_contrast_4,spec_contrast_5,spec_contrast_6,tempo,label
0,-255.973831,110.827522,21.460573,67.588135,4.288312,43.118462,-4.508348,10.442597,-8.718592,1.654890,...,0.027574,25.499474,19.092094,20.626000,17.990107,19.863146,18.939434,35.851431,123.046875,blues
1,-52.701164,74.455414,-18.715458,56.307217,-3.959327,31.987156,-16.457788,22.417690,-12.262029,27.228352,...,0.051619,19.153947,16.769936,21.166412,21.773673,21.574687,19.244753,44.478021,117.453835,blues
2,-118.370331,74.867317,-19.869467,46.947586,-32.569054,23.835136,-26.374111,17.692514,-13.826244,11.946739,...,0.050530,20.249789,16.411930,20.794435,20.532554,20.034756,19.510043,41.900797,107.666016,blues
3,-90.707222,100.107582,-23.924427,52.588303,-20.928015,31.502182,-11.213113,20.094328,-10.915236,17.563923,...,0.049328,19.394646,14.241172,16.221734,17.508593,18.277916,18.680311,41.277834,123.046875,blues
4,-163.243134,89.784889,12.977459,80.692482,3.666775,39.745129,-5.400379,26.498671,-13.163736,2.707013,...,0.016320,24.200335,15.850984,19.495703,17.599240,19.718683,17.326423,39.332214,129.199219,blues


In [ ]:
df.shape

(999, 36)

In [ ]:
np.save('/content/drive/MyDrive/genre_X.npy', X)
np.save('/content/drive/MyDrive/genre_y.npy', Y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
x_dataset = df.drop("label", axis = 1)
y_dataset = df["label"]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    x_dataset, y_dataset,           # input features and labels
    test_size=0.2,  # 20% of data used for testing
    random_state=42,  # for reproducibility
    stratify=y_dataset     # ensures same label distribution in train and test
)


In [38]:
model = RandomForestClassifier(
    n_estimators=100,      # number of trees
    max_depth=None,        # full depth
    random_state=42        # reproducibility
)

In [39]:
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [40]:
accuracy = model.score(X_test, y_test)
print(f"🎯 Test Accuracy: {accuracy:.2f}")

🎯 Test Accuracy: 0.65


In [44]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [43]:
# i am going to try svm
from sklearn.svm import SVC

In [45]:
model = SVC(
    kernel='rbf',          # Gaussian kernel (works well in most cases)
    C=10,                  # Regularization (higher = less margin, more fitting)
    gamma='scale',         # Kernel coefficient (auto-tuned)
    random_state=42
)

model.fit(X_train_scaled, y_train)


SVC(C=10, random_state=42)

In [46]:
accuracy = model.score(X_test_scaled, y_test)
print(f"🎯 SVM Test Accuracy: {accuracy:.2f}")


🎯 SVM Test Accuracy: 0.70


In [47]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

grid = GridSearchCV(SVC(), param_grid, cv=3, verbose=2)
grid.fit(X_train_scaled, y_train)

print("🔍 Best Parameters:", grid.best_params_)
print("🎯 Best Accuracy:", grid.best_score_)


Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END ...................C=0.1, gamma=0.01, kernel=linear; total time=   0.0s
[CV] END ...................C=0.1, gamma=0.01, kernel=linear; total time=   0.0s
[CV] END ...................C=0.1, gamma=0.01, k

In [51]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [57]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier(
    n_estimators=200,
    max_depth=7,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)
model_xgb.fit(X_train_scaled, y_train_encoded)

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [18:12:36] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [61]:
import joblib
joblib.dump(model, 'svm_genre_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [63]:
import numpy as np
np.save('X_features.npy', X)
np.save('y_labels.npy', Y)
